In [1]:

# personal note, CTC Tensorflow documentation is not at all clear and intuitive , so if confused refer to any example

# trying to implement word level CTC

In [1]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
try:
    from tensorflow.python.ops import ctc_ops
except ImportError:
    from tensorflow.contrib.ctc import ctc_ops

In [2]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [3]:
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [4]:
inputs.shape

(299, 13)

In [5]:
train_inputs = np.asarray(inputs[np.newaxis, :])
train_inputs = (train_inputs - np.mean(train_inputs))/np.std(train_inputs)
train_seq_len = [train_inputs.shape[1]]
print(train_inputs.shape)

(1, 299, 13)


In [6]:
# we also need a fixed vocabulary 
import re

word_dictionary = {'hello':0 , 'world':1 , ' ':2}


def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
        if word in word_dictionary:
           # print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list
  

In [7]:
# this function is required for CTC Loss
# for it's input , first convert transcrition / ground truth to number representation 

def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

In [8]:
output_transcript = word_to_index('hello world hello world')
batch_list = []
batch_list.insert(0,output_transcript)
train_targets = sparse_tuple_from(batch_list)

In [9]:
# Training Parameters
learning_rate = 0.001
training_steps = 10000
batch_size = 1
display_step = 200
num_features = 13

# Network Parameters
num_input = 28 # MNIST data input (img shape: 28*28)
timesteps = 299 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 4 # hello , world , blank and space 

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])


In [10]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([batch_size, 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [11]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    #convert output shape (timesteps * batch * classes ) to (batch*timesteps*classes)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    print('shape of outputs as produced by blstm ')
    print(outputs.shape)
    
    #temp=tf.convert_to_tensor(outputs)[:,-1,:]
    
   # temp=temp[:,-1,:]
    #return outputs
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    print('shape of result ')
    print(res.shape)
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [12]:

#prediction = tf.nn.softmax(logits)
inputs = tf.placeholder(tf.float32, [None, None, num_features])
targets = tf.sparse_placeholder(tf.int32)
seq_len = tf.placeholder(tf.int32, [None])

logits = RNN(inputs, weights, biases)

loss = ctc_ops.ctc_loss( targets, logits , seq_len)
cost = tf.reduce_mean(loss)
optimizer = tf.train.MomentumOptimizer(learning_rate,
                                           0.9).minimize(cost)


shape of outputs as produced by blstm 
(?, 299, 256)
shape of result 
(1, 299, 4)


In [13]:
init = tf.global_variables_initializer()

In [14]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
    
    
    print(train_seq_len)
    
    feed = {inputs: train_inputs,
                    targets: train_targets,
                    seq_len: train_seq_len}
	    

    batch_cost, _ = sess.run([cost, optimizer], feed)
    train_cost += batch_cost*batch_size
    

[299]


InvalidArgumentError: len(sequence_length) != batch_size.  len(sequence_length):  1 batch_size: 299
	 [[Node: CTCLoss = CTCLoss[ctc_merge_repeated=true, ignore_longer_outputs_than_inputs=false, preprocess_collapse_repeated=false, _device="/job:localhost/replica:0/task:0/cpu:0"](add, _arg_Placeholder_5_0_3, _arg_Placeholder_4_0_2, _arg_Placeholder_6_0_4)]]

Caused by op 'CTCLoss', defined at:
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/saurabh/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/saurabh/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    